<a href="https://colab.research.google.com/github/HarshKapr/Cloud/blob/main/Assesment2_SIG742_Harsh_Pavan_Team19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>